In [20]:
import pandas as pd
import numpy as np

Load the data from CSV files

In [21]:
foundational_percentages = pd.read_csv('../data/labor/foundational_percentages.csv')
labor_hours = pd.read_csv('../data/labor/lc_nnum1_r2_linear.csv')
energy_use = pd.read_csv('../data/biophysical/env_ac_pefa04_linear.csv')

Merge the two tables on nace_2digits column, keeping columns from labor_hours and adding column final_foundational from foundational_percentages

In [22]:
foundational_percentages_selected = foundational_percentages[['nace_r2', 'final_foundational']]

In [23]:
labor_hours_with_foundational = labor_hours.merge(foundational_percentages_selected, on='nace_r2', how='left')
energy_use_with_foundational = energy_use.merge(foundational_percentages_selected, on='nace_r2', how='left')